In [11]:
import os
import shutil

kaggle_token_path = '/content/drive/MyDrive/kaggle/kaggle.json'

colab_token_dir = '/root/.kaggle/'

if not os.path.exists(colab_token_dir):
    os.makedirs(colab_token_dir)

colab_token_path = os.path.join(colab_token_dir, 'kaggle.json')

shutil.copyfile(kaggle_token_path, colab_token_path)

'/root/.kaggle/kaggle.json'

In [12]:
from google.colab import files

uploaded = files.upload()


!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.js/
!kaggle datasets download -d akash2907/bird-species-classification
!unzip bird-species-classification.zip

chmod: cannot access '/root/.kaggle/kaggle.js/': No such file or directory
100% 1.37G/1.37G [00:08<00:00, 226MB/s]
100% 1.37G/1.37G [00:08<00:00, 184MB/s]
Archive:  bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/

In [13]:
import numpy as np
import pandas as pd
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential

import numpy as np
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import EarlyStopping

In [14]:
root_train = '/content/train_data/train_data'
root_test = '/content/test_data/test_data'

train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/test_data/test_data',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)

train.class_indices

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [15]:
model = Sequential()
# Adding conv layer with input
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the conv layer output
model.add(BatchNormalization())
# Selecting the max values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hiddern layers
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(16,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 12)      336       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 12)     48        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 12)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 24)       9

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [18]:
model.fit(train,batch_size=8,validation_data=test,epochs=50,callbacks=early_stop)
# model.fit(train,batch_size=8,validation_data=test,epochs=50)

Epoch 1/50
19/19 [==============================] - 110s 6s/step - loss: 2.8501 - accuracy: 0.1533 - val_loss: 2.8011 - val_accuracy: 0.0573
Epoch 2/50
19/19 [==============================] - 127s 7s/step - loss: 2.5621 - accuracy: 0.1933 - val_loss: 2.9037 - val_accuracy: 0.0573
Epoch 3/50
19/19 [==============================] - 127s 7s/step - loss: 2.5002 - accuracy: 0.2000 - val_loss: 2.9098 - val_accuracy: 0.0573
Epoch 4/50
19/19 [==============================] - 108s 6s/step - loss: 2.4168 - accuracy: 0.2400 - val_loss: 3.0335 - val_accuracy: 0.0573
Epoch 5/50
19/19 [==============================] - 128s 7s/step - loss: 2.3632 - accuracy: 0.1933 - val_loss: 3.6845 - val_accuracy: 0.0573
Epoch 6/50
19/19 [==============================] - 127s 7s/step - loss: 2.3196 - accuracy: 0.2733 - val_loss: 4.1059 - val_accuracy: 0.0573
Epoch 7/50
19/19 [==============================] - 127s 7s/step - loss: 2.1793 - accuracy: 0.3200 - val_loss: 3.7985 - val_accuracy: 0.0828
Epoch 8/50
19

In [19]:
model.save('birds.h5')

In [20]:
img1 = image.load_img('/content/test_data/test_data/bonegl/DSC_4587.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti', 'bonegl', "brhkyt", "cbrtsh", "cmnmyn", "gretit", "hilpig", "himbul",
          "himgri", "hsparo", "indvul", "jglowl", "lbicrw", "mgprob", "rebimg", "wcrsrt"]
print(output[pred])

1/1 [==============================] - 0s 496ms/step
4
cmnmyn


In [21]:
model.predict(test)

20/20 [==============================] - 60s 3s/step


array([[4.8989486e-03, 6.4469503e-05, 5.3983698e-05, ..., 4.9450330e-02,
        1.1746264e-02, 1.5079689e-05],
       [7.5050652e-02, 1.4831240e-01, 8.5101230e-03, ..., 1.3834612e-03,
        1.1357262e-04, 2.4111151e-04],
       [4.2047971e-01, 1.9121199e-04, 1.6438544e-06, ..., 1.7645885e-03,
        9.3445859e-05, 8.2658389e-06],
       ...,
       [7.1953580e-02, 5.7760376e-02, 1.7527539e-03, ..., 1.2059783e-04,
        9.7801421e-05, 9.5501782e-05],
       [1.6097364e-01, 1.6097270e-03, 4.8550777e-05, ..., 5.1856558e-03,
        1.4793311e-04, 8.3845349e-05],
       [7.0731968e-02, 5.0472782e-04, 3.0570136e-05, ..., 7.9771513e-01,
        1.2499515e-02, 9.2090043e-04]], dtype=float32)

In [22]:
test_loss, test_accuracy = model.evaluate(test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

20/20 [==============================] - 58s 3s/step - loss: 3.9644 - accuracy: 0.1656
Test Loss: 3.9643845558166504
Test Accuracy: 0.1656050980091095
